![](../../../../Documents/Screenshots/screenshot.286.jpg)
![](../../../../Documents/Screenshots/screenshot.284.jpg)


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.python.ops.init_ops_v2 import glorot_uniform
from keras.layers import ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout
from keras import Sequential
import tensorflow as tf
import pickle

In [3]:


shape = (96, 96, 1)


def convolutional_block(filters):
    f1, f2, f3 = filters

    return [
        Conv2D(f1, (1, 1), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0)),
        MaxPooling2D((2, 2)),
        BatchNormalization(axis=3),
        Activation('relu'),

        Conv2D(f2, (3, 3), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0)),
        BatchNormalization(axis=3),
        Activation('relu'),

        Conv2D(f3, (3, 3), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0)),
        BatchNormalization(axis=3),

        Conv2D(f3, (1, 1), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0)),
        MaxPooling2D((2, 2)),
        BatchNormalization(axis=3),

        Activation('relu')
    ]


def identity_block(filters):
    f1, f2, f3 = filters

    return [
        Conv2D(f1, (1, 1), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0)),
        BatchNormalization(axis=3),
        Activation('relu'),

        Conv2D(f2, (3, 3), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0)),
        BatchNormalization(axis=3),
        Activation('relu'),

        Conv2D(f3, (1, 1), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0)),
        BatchNormalization(axis=3),
        Activation('relu')
    ]


def res_block(model, filters):
    layers = []

    layers.extend(convolutional_block(filters))
    layers.extend(identity_block(filters))
    layers.extend(identity_block(filters))

    for layer in layers:
        model.add(layer)


def resnet():
    model = Sequential()
    model.add(ZeroPadding2D((3, 3), input_shape=shape))
    model.add(Conv2D(64, (3, 3), name='conv1'))
    model.add(BatchNormalization(axis=3, name='bn_conv1'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((3, 3)))

    res_block(model, [64, 64, 256])
    res_block(model, [128, 128, 512])

    model.add(AveragePooling2D((2, 2), name='avg_pool'))
    model.add(Flatten())

    model.add(Dense(4096, activation='relu'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2048, activation='relu'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(30, activation='relu'))

    return model

In [4]:
model = resnet()
model.summary()

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 102, 102, 1)      0         
 2D)                                                             
                                                                 
 conv1 (Conv2D)              (None, 100, 100, 64)      640       
                                                                 
 bn_conv1 (BatchNormalizatio  (None, 100, 100, 64)     256       
 n)                                                              
                                                                 
 activation (Activation)     (None, 100, 100, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 33, 33, 64)       0         
 )                                                               
                                                        

In [6]:
data = pd.read_feather('datasets/augmented_data.feather').copy()
data

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,67.671771,34.955794,30.898286,34.036457,61.849371,35.262309,73.188343,35.875200,37.333714,35.262309,...,53.649600,62.155886,76.020343,35.494629,74.794286,48.365486,71.116800,48.365486,80.923200,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
1,31.254295,35.921025,61.864642,39.444518,36.979972,37.484575,26.057143,35.260370,57.019839,39.290365,...,52.060657,35.134884,75.526531,59.572169,76.397502,41.777883,75.711058,43.131087,78.807024,0.4014 0.3982 0.3997 0.3982 0.3901 0.3801 0.38...
2,64.370699,60.584782,58.726033,28.572293,62.453406,53.210688,65.883418,67.413173,58.584578,34.766968,...,46.151078,28.029345,62.673796,24.706874,38.581047,29.492332,48.225583,19.530965,51.216123,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
3,67.430201,38.067488,29.395599,39.855280,59.811322,39.393218,75.069343,39.042768,38.021315,40.347239,...,59.035017,70.786215,72.635128,31.340844,76.293426,51.013536,70.575612,52.151253,84.066436,0.002028 0.0007954 0.0005965 0.0005965 0.00067...
4,65.304107,36.543259,28.844745,37.428173,57.284578,39.391575,74.014975,37.677055,38.717063,39.723418,...,60.757739,65.176153,72.101105,29.609314,73.965298,48.338648,75.323869,48.398417,81.397541,0.001878 0.00202 0.002743 0.002092 0.001203 0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42795,61.780769,30.876849,23.146882,36.066028,54.332347,33.765992,71.931763,30.067577,34.192934,36.880099,...,61.106977,66.946179,78.849739,32.860750,83.545325,52.802848,82.753299,53.198861,83.432178,0.0002618 0.0001496 0.0001122 0.0001869 0.0002...
42796,61.713259,32.374178,28.300633,38.780817,55.727903,34.975987,68.139900,32.271748,35.020965,38.885013,...,52.852166,66.591061,67.345551,37.472421,73.252879,51.340933,65.651955,53.354387,75.608568,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0....
42797,70.205428,36.172943,29.954357,29.227776,62.532826,35.582372,79.000536,38.181007,38.497192,31.801490,...,56.575488,58.511053,77.416298,26.498234,71.313974,42.799900,70.529005,41.357206,79.575985,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0004797 0.00...
42798,40.323722,28.643860,32.585601,62.538858,39.350782,35.928832,41.711117,20.904723,34.465993,56.134493,...,49.518453,78.541504,38.006918,71.528758,67.465329,75.978338,53.020691,76.705854,53.548766,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.001392 0.001...


In [ ]:
from sklearn.model_selection import train_test_split

data_in_batches = np.array_split(data, 50)

train_xs = []
train_ys = []
test_xs = []
test_ys = []

index = 0
for batch in data_in_batches:
    images = batch['Image'].str.split(' ', expand=True).astype(np.float32).to_numpy().reshape(-1, 96, 96, 1)
    points = batch.iloc[:, :-1].to_numpy()

    train_x, test_x, train_y, test_y = train_test_split(images, points, train_size=0.8)
    train_xs.extend(train_x.tolist())
    train_ys.extend(train_y.tolist())
    test_xs.extend(test_x.tolist())
    test_ys.extend(test_y.tolist())

    index += 1
    print(index)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [ ]:
model.train_on_batch(train_xs, train_ys, epochs=200, validation_data=(test_xs, test_ys))
model.save('models/facialPointsPredictor.h5')